In [ ]:
# 1. Import các thư viện cần thiết
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm  # Hiển thị progress bar khi train/test

In [ ]:
# 2. Chọn thiết bị (GPU nếu có, CPU nếu không)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# 3. Đường dẫn tới thư mục chứa dữ liệu bạn vừa tải về
data_dir = './fruits-fresh-and-rotten-for-classification/dataset'  # chỉnh lại nếu khác

In [ ]:
# 4. Định nghĩa transform cho dữ liệu train và test
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Cắt ngẫu nhiên và resize về 224x224
        transforms.RandomHorizontalFlip(p=0.5),                # Lật ngang ngẫu nhiên
        transforms.RandomRotation(degrees=15),                 # Xoay ảnh ±15 độ
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Thay đổi độ sáng, tương phản, màu sắc
        transforms.ToTensor(),                                  # Chuyển ảnh sang tensor
        transforms.Normalize([0.485, 0.456, 0.406],            # Chuẩn hóa theo mean của ImageNet
                             [0.229, 0.224, 0.225])            # Chuẩn hóa theo std của ImageNet
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),                         # Resize ảnh về 224x224
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
# 5. Tạo dataset cho train và test sử dụng ImageFolder
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'test']
}

In [ ]:
# 6. Tạo DataLoader để load dữ liệu theo batch
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=64, shuffle=True, num_workers=2),
    'test': DataLoader(image_datasets['test'], batch_size=64, shuffle=False, num_workers=2)
}

In [ ]:
# 7. Lấy tên các lớp (label)
class_names = image_datasets['train'].classes
print("Classes:", class_names)


In [ ]:
# 8. Xây dựng mô hình CNN từ đầu(optional, độ chính xác sẽ thấp hơn)
model = nn.Sequential(
    # Lớp conv 1: 3 kênh đầu vào (RGB), 64 filter, kernel 3x3
    nn.Conv2d(3, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),  # Chuẩn hóa batch
    nn.ReLU(),           # Hàm kích hoạt ReLU
    nn.MaxPool2d(2, 2),  # Giảm kích thước ảnh đi 2 lần

    # Lớp conv 2
    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    # Lớp conv 3
    nn.Conv2d(128, 256, kernel_size=3, padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    # Lớp conv 4
    nn.Conv2d(256, 512, kernel_size=3, padding=1),
    nn.BatchNorm2d(512),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    nn.Flatten(),  # Chuyển tensor 4D thành 2D (batch_size, features)

    # Fully connected layer 1
    nn.Linear(512 * 14 * 14, 512),  # 14x14 là kích thước feature map sau conv+pooling (224/2/2/2/2)
    nn.ReLU(),
    nn.Dropout(0.3),

    # Fully connected layer 2 (output)
    nn.Linear(512, len(class_names))  # Số output bằng số lớp
)

# Chuyển model lên GPU nếu có
model = model.to(device)

In [ ]:
#Sử dụng mô hình pretrained
# 6. Load pretrained ResNet18
model = models.resnet18(pretrained=True)

# 7. Thay thế lớp fully connected cuối cùng cho phù hợp số lớp bài toán
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

# 8. Chuyển model lên GPU
model = model.to(device)

In [ ]:
# 9. Định nghĩa hàm loss và optimizer
criterion = nn.CrossEntropyLoss()  # Phù hợp bài toán phân loại đa lớp
optimizer = optim.Adam(model.parameters(), lr=1e-3)  # Adam optimizer với learning rate 0.001

In [ ]:
# 10. Hàm đánh giá accuracy trên tập test
def evaluate(model, dataloader):
    model.eval()  # Chế độ đánh giá
    correct = 0
    total = 0
    with torch.no_grad():  # Tắt tính gradient
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)  # Lấy nhãn dự đoán
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = correct / total
    return acc

# 11. Vòng lặp huấn luyện
num_epochs = 15
for epoch in range(num_epochs):
    model.train()  # Chế độ train
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    for inputs, labels in tqdm(dataloaders['train'], desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()  # Reset gradient
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Tính loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Cập nhật trọng số

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += (preds == labels).sum().item()
        total_samples += inputs.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects / total_samples
    val_acc = evaluate(model, dataloaders['test'])

    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Train Acc: {epoch_acc:.4f} - Val Acc: {val_acc:.4f}")
    

In [ ]:
#Lưu lại model
torch.save(model.state_dict(), '/kaggle/working/fruit_classifier_resnet18.pth')